In [34]:
library(tidyverse)
library(magrittr)
library(htmlwidgets)
library(IRdisplay)
library(leaflet)

In [24]:
aeroportos = file.path(path,"airports.csv")
colsairports = cols_only(IATA_CODE = "c", CITY = "c", STATE = "c", LATITUDE = "d", LONGITUDE = "d")
airports = read_csv(aeroportos, col_types = colsairports)

In [25]:
head(airports)

IATA_CODE,CITY,STATE,LATITUDE,LONGITUDE
<chr>,<chr>,<chr>,<dbl>,<dbl>
ABE,Allentown,PA,40.65236,-75.44040
ABI,Abilene,TX,32.41132,-99.68190
ABQ,Albuquerque,NM,35.04022,-106.60919
ABR,Aberdeen,SD,45.44906,-98.42183
ABY,Albany,GA,31.53552,-84.19447
ACK,Nantucket,MA,41.25305,-70.06018


In [26]:
voos = file.path(path, "flights_small.csv.zip")
colsflights = cols_only(DESTINATION_AIRPORT = "c", ARRIVAL_DELAY = "i")
getStats = function(input, pos){
    input  %>% filter(!startsWith(DESTINATION_AIRPORT, "1")) %>% 
    drop_na() %>%
    group_by(DESTINATION_AIRPORT) %>% 
    summarise(Sx = sum(ARRIVAL_DELAY), n = n())    
}
## No lugar ddrop_na pode usar filter(!is.na(DESTINATION_AIRPORT), !is.na(ARRIVAL_DELAY)) ou na.omit)()

flights = read_csv_chunked(voos, chunk_size = 1000, callback = DataFrameCallback$new(getStats), col_types = colsflights)

In [27]:
##x = x %>% filter(bla) é igual a 
##x =  %<>% filter(bla)

flights  %<>% group_by(DESTINATION_AIRPORT) %>% summarise(ATRASO_MEDIO = sum(Sx)/sum(n))

In [28]:
flights %>% head

DESTINATION_AIRPORT,ATRASO_MEDIO
<chr>,<dbl>
ABE,-5.250000
ABI,52.000000
ABQ,9.115385
ABR,-11.000000
ABY,-3.500000
ACT,79.500000


In [30]:
flights %<>% left_join(airports, by = c("DESTINATION_AIRPORT" = "IATA_CODE"))

In [31]:
flights %>% head

DESTINATION_AIRPORT,ATRASO_MEDIO,CITY,STATE,LATITUDE,LONGITUDE
<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
ABE,-5.250000,Allentown,PA,40.65236,-75.44040
ABI,52.000000,Abilene,TX,32.41132,-99.68190
ABQ,9.115385,Albuquerque,NM,35.04022,-106.60919
ABR,-11.000000,Aberdeen,SD,45.44906,-98.42183
ABY,-3.500000,Albany,GA,31.53552,-84.19447
ACT,79.500000,Waco,TX,31.61129,-97.23052


In [2]:
(estados = flights %>% group_by(STATE) %>% count() %>% arrange(desc(n)))

ERROR: Error in flights %>% group_by(STATE) %>% count() %>% arrange(desc(n)): could not find function "%>%"


In [54]:
this = leaflet(flights) %>%
    addTiles() %>%
        addCircleMarkers(~LONGITUDE, ~LATITUDE,
                        radius = ~ATRASO_MEDIO,
                        fillOpacity = 0.5,
                        clusterOptions = markerClusterOptions(),
                        label = ~as.character(DESTINATION_AIRPORT),
                        popup = ~as.character(paste("O atraso é de",round(ATRASO_MEDIO,0),"minutos")))
saveWidget(this, 'demo.html', selfcontained = FALSE)
display_html('<iframe src="demo.html" width="600" height="600"></iframe>')

Warning message in validateCoords(lng, lat, funcName):
“Data contains 2 rows with either missing or invalid lat/lon values and will be ignored”